In [ ]:
from functions import download_youtube
video_id = 'ek05M8eCk7M'
#video_id = 'I-cxigjLG0I'
youtube_url = f"https://www.youtube.com/watch?v={video_id}"
job_id = 'Hanni'
file_path, file_name =  download_youtube(youtube_url, job_id=job_id)
file_name

In [ ]:
from functions import slice_video
slice_video(file_name, [(0, 240, 'hanni.mp4')], height=540, width=960)

### OpenCV 설치
- `pip uninstall opencv-python`
- `pip install opencv-contrib-python`

### Face Detection & Recognition

In [ ]:
base_image = "../face_recognition/test_hanni2.jpg"
target_video = "hanni.mp4"

import cv2
import face_recognition

# 특정 인물의 이미지를 로드하여 인코딩
known_image = face_recognition.load_image_file(base_image)
known_face_encoding = face_recognition.face_encodings(known_image)[0]

# 비디오 파일 로드
video_capture = cv2.VideoCapture(target_video)

# 변수 초기화
trackers = []
face_names = []
face_distances = []
frame_skip = 12
frame_count = 0

def expand_bounding_box(left, top, width, height, frame_width, frame_height, expand_ratio=0.25):
    expand_width = int(width * expand_ratio)
    expand_height = int(height * expand_ratio)
    
    expanded_left = int(max(0, left - expand_width))
    expanded_top = int(max(0, top - expand_height))
    expanded_right = int(min(frame_width, left + width + expand_width))
    expanded_bottom = int(min(frame_height, top + height + expand_height))
    
    return expanded_left, expanded_top, expanded_right, expanded_bottom

def draw_face_annotations(frame, name, distance, bbox, frame_width, frame_height):
    left, top, width, height = bbox
    expanded_left, expanded_top, expanded_right, expanded_bottom = expand_bounding_box(
        left, top, width, height, frame_width, frame_height)
    
    # 좌표 값을 정수형으로 변환
    expanded_left = int(expanded_left)
    expanded_top = int(expanded_top)
    expanded_right = int(expanded_right)
    expanded_bottom = int(expanded_bottom)
    
    # 이름에 따른 색상 설정
    if name == "Specific Person":
        color = (0, 0, 255)  # 빨간색
    elif name == "Candidate":
        color = (255, 0, 0)  # 파란색
    else:
        color = (0, 255, 0)  # 초록색
    
    # 사각형 그리기
    cv2.rectangle(frame, (expanded_left, expanded_top), (expanded_right, expanded_bottom), color, 2)
    
    # 이름과 거리 표시
    cv2.putText(frame, name, (expanded_left, expanded_bottom + 20), cv2.FONT_HERSHEY_SIMPLEX, 0.6, color, 2)
    cv2.putText(frame, f"Distance: {distance:.2f}", (expanded_left, expanded_bottom + 45), cv2.FONT_HERSHEY_SIMPLEX, 0.6, color, 2)

while True:
    # 비디오에서 한 프레임씩 읽기
    ret, frame = video_capture.read()

    if not ret:
        break

    frame_count += 1

    rgb_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)

    # 프레임의 높이와 너비를 가져오기
    frame_height, frame_width, _ = frame.shape

    if frame_count % frame_skip == 0:
        # 추적기와 얼굴 정보 초기화
        trackers = []
        face_names = []
        face_distances = []

        # 얼굴 위치와 인코딩 탐지
        face_locations = face_recognition.face_locations(rgb_frame)
        face_encodings = face_recognition.face_encodings(rgb_frame, face_locations)

        distances = []
        for face_encoding in face_encodings:
            # 얼굴 거리 계산
            distance = face_recognition.face_distance([known_face_encoding], face_encoding)[0]
            distances.append(distance)

        # 임계값 설정 (예: 0.46)
        tolerance = 0.46

        # 거리 기준으로 가장 작은 인덱스 찾기
        specific_person_index = None
        if len(distances) > 0:
            min_distance = min(distances)
            if min_distance < tolerance:
                specific_person_index = distances.index(min_distance)

        for idx, (face_location, face_encoding) in enumerate(zip(face_locations, face_encodings)):
            top, right, bottom, left = face_location

            # 얼굴 거리 계산
            distance = distances[idx]

            if distance < tolerance:
                if idx == specific_person_index:
                    name = "Specific Person"
                else:
                    name = "Candidate"
            else:
                name = "Unknown"

            # 추적기 초기화
            tracker = cv2.legacy.TrackerKCF_create()
            bbox = (left, top, right - left, bottom - top)
            tracker.init(frame, bbox)
            trackers.append(tracker)
            face_names.append(name)
            face_distances.append(distance)

            # 얼굴 주석 그리기
            draw_face_annotations(frame, name, distance, bbox, frame_width, frame_height)

    else:
        # 모든 추적기 업데이트
        new_trackers = []
        new_face_names = []
        new_face_distances = []

        for tracker, name, distance in zip(trackers, face_names, face_distances):
            success, bbox = tracker.update(frame)
            if success:
                new_trackers.append(tracker)
                new_face_names.append(name)
                new_face_distances.append(distance)

                # 좌표 값을 정수형으로 변환
                bbox = tuple(map(int, bbox))

                # 얼굴 주석 그리기
                draw_face_annotations(frame, name, distance, bbox, frame_width, frame_height)

        # 추적기와 얼굴 정보 업데이트
        trackers = new_trackers
        face_names = new_face_names
        face_distances = new_face_distances

    # 비디오 출력
    cv2.imshow('Video', frame)

    # 'q' 키를 누르면 종료
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

# 모든 작업 완료 후 클린업
video_capture.release()
cv2.destroyAllWindows()

### Stable Diffusion Web UI API 를 사용하여 Face Swapping

In [ ]:
base_image = "../face_recognition/test_hanni2.jpg"
source_image = "../face_recognition/faces/김태희.jpg"
target_video = "hanni.mp4"

import io
import cv2
import face_recognition
import numpy as np
from get_swapped_face import get_swapped_face

# 특정 인물의 이미지를 로드하여 인코딩
known_image = face_recognition.load_image_file(base_image)
known_face_encoding = face_recognition.face_encodings(known_image)[0]

with open(source_image, "rb") as source_file:
    source_bytes = source_file.read()

# 비디오 파일 로드
video_capture = cv2.VideoCapture(target_video)

# 변수 초기화
trackers = []
face_names = []
face_distances = []
frame_skip = 12
frame_count = 0

# swap_face 함수가 이미 존재한다고 가정합니다.
# def swap_face(jpg_binary):
#     # 얼굴 스왑 처리 후 동일한 크기의 jpg_binary 반환
#     return swapped_jpg_binary

while True:
    # 비디오에서 한 프레임씩 읽기
    ret, frame = video_capture.read()

    if not ret:
        break

    frame_count += 1

    rgb_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)

    # 프레임의 높이와 너비를 가져오기
    frame_height, frame_width, _ = frame.shape

    if frame_count % frame_skip == 0:
        # 추적기와 얼굴 정보 초기화
        trackers = []
        face_names = []
        face_distances = []

        # 얼굴 위치와 인코딩 탐지
        face_locations = face_recognition.face_locations(rgb_frame)
        face_encodings = face_recognition.face_encodings(rgb_frame, face_locations)

        distances = []
        for face_encoding in face_encodings:
            # 얼굴 거리 계산
            distance = face_recognition.face_distance([known_face_encoding], face_encoding)[0]
            distances.append(distance)

        # 임계값 설정 (예: 0.46)
        tolerance = 0.46

        # 거리 기준으로 가장 작은 인덱스 찾기
        specific_person_index = None
        if len(distances) > 0:
            min_distance = min(distances)
            if min_distance < tolerance:
                specific_person_index = distances.index(min_distance)

        for idx, (face_location, face_encoding) in enumerate(zip(face_locations, face_encodings)):
            top, right, bottom, left = face_location

            # 얼굴 거리 계산
            distance = distances[idx]

            if distance < tolerance:
                if idx == specific_person_index:
                    name = "Specific Person"
                else:
                    name = "Candidate"
            else:
                name = "Unknown"

            # 추적기 초기화
            tracker = cv2.legacy.TrackerKCF_create()
            bbox = (left, top, right - left, bottom - top)
            tracker.init(frame, bbox)
            trackers.append(tracker)
            face_names.append(name)
            face_distances.append(distance)
    else:
        # 모든 추적기 업데이트
        new_trackers = []
        new_face_names = []
        new_face_distances = []

        for tracker, name, distance in zip(trackers, face_names, face_distances):
            success, bbox = tracker.update(frame)
            if success:
                new_trackers.append(tracker)
                new_face_names.append(name)
                new_face_distances.append(distance)
                left, top, width, height = [int(v) for v in bbox]
                right = left + width
                bottom = top + height

                # 사각형을 4방으로 1.5배 확장
                expand_width = int(width * 0.25)  # 원래의 0.25배씩 좌우로 확장
                expand_height = int(height * 0.25)  # 원래의 0.25배씩 상하로 확장

                # 새로운 좌표 계산 (프레임 경계를 넘지 않도록 조정)
                expanded_left = max(0, left - expand_width)
                expanded_top = max(0, top - expand_height)
                expanded_right = min(frame_width, right + expand_width)
                expanded_bottom = min(frame_height, bottom + expand_height)

                # 사각형과 텍스트 표시
                if name == "Specific Person":
                    # 특정 인물에 대해 얼굴 스왑 처리

                    # 확장된 사각형 영역 추출
                    face_region = frame[expanded_top:expanded_bottom, expanded_left:expanded_right]

                    # 이미지를 JPG 바이너리로 인코딩
                    success_enc, jpg_buffer = cv2.imencode('.jpg', face_region)
                    if success_enc:
                        jpg_binary = jpg_buffer.tobytes()

                        # swap_face 함수 호출
                        swap_png_binary = get_swapped_face(source_bytes, jpg_binary)

                        # swap_png_binary가 BytesIO 객체인 경우 처리
                        if isinstance(swap_png_binary, io.BytesIO):
                            swap_png_binary = swap_png_binary.getvalue()

                        # swap_png_binary를 이미지로 디코딩
                        swap_image_array = np.frombuffer(swap_png_binary, np.uint8)
                        swap_image = cv2.imdecode(swap_image_array, cv2.IMREAD_COLOR)

                        # 스왑된 이미지의 크기가 영역과 다를 수 있으므로 리사이즈
                        swap_image = cv2.resize(swap_image, (expanded_right - expanded_left, expanded_bottom - expanded_top))

                        # 스왑된 이미지를 프레임에 적용
                        frame[expanded_top:expanded_bottom, expanded_left:expanded_right] = swap_image

                    # 사각형 그리기
                    color = (0, 0, 255)  # 빨간색
                    cv2.rectangle(frame, (expanded_left, expanded_top), (expanded_right, expanded_bottom), color, 2)
                    # 이름 표시
                    cv2.putText(frame, name, (expanded_left, expanded_bottom + 20), cv2.FONT_HERSHEY_SIMPLEX, 0.6, color, 2)
                    # 거리 표시 (이름 아래)
                    cv2.putText(frame, f"Distance: {distance:.2f}", (expanded_left, expanded_bottom + 45), cv2.FONT_HERSHEY_SIMPLEX, 0.6, color, 2)

                elif name == "Candidate":
                    # 후보자는 파란색 사각형과 이름 표시
                    color = (255, 0, 0)
                    cv2.rectangle(frame, (expanded_left, expanded_top), (expanded_right, expanded_bottom), color, 2)
                    cv2.putText(frame, name, (expanded_left, expanded_bottom + 20), cv2.FONT_HERSHEY_SIMPLEX, 0.6, color, 2)
                    cv2.putText(frame, f"Distance: {distance:.2f}", (expanded_left, expanded_bottom + 45), cv2.FONT_HERSHEY_SIMPLEX, 0.6, color, 2)
                else:
                    # 다른 인물에 대해 초록색 사각형과 이름만 표시
                    color = (0, 255, 0)
                    cv2.rectangle(frame, (expanded_left, expanded_top), (expanded_right, expanded_bottom), color, 2)
                    cv2.putText(frame, name, (expanded_left, expanded_bottom + 20), cv2.FONT_HERSHEY_SIMPLEX, 0.6, color, 2)
                    cv2.putText(frame, f"Distance: {distance:.2f}", (expanded_left, expanded_bottom + 45), cv2.FONT_HERSHEY_SIMPLEX, 0.6, color, 2)

        # 추적기와 얼굴 정보 업데이트
        trackers = new_trackers
        face_names = new_face_names
        face_distances = new_face_distances

    # 비디오 출력
    cv2.imshow('Video', frame)

    # 'q' 키를 누르면 종료
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

# 모든 작업 완료 후 클린업
video_capture.release()
cv2.destroyAllWindows()


### insightface library 를 사용하여 Face Swapping

```
    pip install insightface
    pip install onnxruntime #for CPU-only
    pip install onnxruntime-gpu #For GPU
    pip uninstall opencv-python-headless
    pip install opencv-contrib-python-headless # 또는 opencv-contrib-python
```
- Download `inswapper_128.onnx` & Locate it in a specific directory
- buffalo_l download from : 
    https://github.com/deepinsight/insightface/releases

- unzip buffalo_l.zip on `C:\Users\<user>\.insightface\models\buffalo_l`

In [ ]:
import io
import cv2
import face_recognition
import numpy as np
from inswapper import FaceSwapper

base_image = "../face_recognition/test_hanni2.jpg"
source_image = "../face_recognition/faces/yunsy.jpg"
target_video = "hanni.mp4"

# 특정 인물의 이미지를 로드하여 인코딩
known_image = face_recognition.load_image_file(base_image)
known_face_encoding = face_recognition.face_encodings(known_image)[0]

# FaceSwapper 클래스 인스턴스 생성
face_swapper = FaceSwapper(det_size=(160, 160))

# 소스 얼굴 설정 (face_index는 선택 사항)
success = face_swapper.set_source_face(source_image)

if not success:
    print("소스 얼굴 설정에 실패했습니다.")
    exit()

# 비디오 파일 로드
video_capture = cv2.VideoCapture(target_video)

# 변수 초기화
trackers = []
face_names = []
face_distances = []
frame_skip = 12
frame_count = 0

def process_face(frame, name, distance, bbox, frame_width, frame_height):
    left, top, width, height = bbox
    expand_ratio=0.3
    expand_width = int(width * expand_ratio)
    expand_height = int(height * expand_ratio)

    expanded_left = int(max(0, left - expand_width))
    expanded_top = int(max(0, top - expand_height))
    expanded_right = int(min(frame_width, left + width + expand_width))
    expanded_bottom = int(min(frame_height, top + height + expand_height))

    # 얼굴 영역 추출
    face_region = frame[expanded_top:expanded_bottom, expanded_left:expanded_right]

    if name == "Specific Person":
        # 얼굴 스왑 처리
        swap_image = face_swapper.swap_faces_in_image(face_region)
        if swap_image is not None:
            # 스왑된 이미지를 얼굴 영역 크기에 맞게 조정
            swap_image = cv2.resize(swap_image, (expanded_right - expanded_left, expanded_bottom - expanded_top))
            # 스왑된 얼굴을 프레임에 적용
            frame[expanded_top:expanded_bottom, expanded_left:expanded_right] = swap_image
        color = (0, 0, 255)  # 빨간색
    elif name == "Candidate":
        color = (255, 0, 0)  # 파란색
    else:
        color = (0, 255, 0)  # 초록색

    # 사각형 그리기
    cv2.rectangle(frame, (expanded_left, expanded_top), (expanded_right, expanded_bottom), color, 2)
    # 이름과 거리 표시
    cv2.putText(frame, name, (expanded_left, expanded_bottom + 20), cv2.FONT_HERSHEY_SIMPLEX, 0.6, color, 2)
    cv2.putText(frame, f"Distance: {distance:.2f}", (expanded_left, expanded_bottom + 45), cv2.FONT_HERSHEY_SIMPLEX, 0.6, color, 2)

while True:
    # 비디오에서 한 프레임씩 읽기
    ret, frame = video_capture.read()

    if not ret:
        break

    frame_count += 1

    rgb_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)

    # 프레임의 높이와 너비를 가져오기
    frame_height, frame_width, _ = frame.shape

    if frame_count % frame_skip == 0:
        # 추적기와 얼굴 정보 초기화
        trackers = []
        face_names = []
        face_distances = []

        # 얼굴 위치와 인코딩 탐지
        face_locations = face_recognition.face_locations(rgb_frame)
        face_encodings = face_recognition.face_encodings(rgb_frame, face_locations)

        distances = []
        for face_encoding in face_encodings:
            # 얼굴 거리 계산
            distance = face_recognition.face_distance([known_face_encoding], face_encoding)[0]
            distances.append(distance)

        # 임계값 설정 (예: 0.46)
        tolerance = 0.46

        # 거리 기준으로 가장 작은 인덱스 찾기
        specific_person_index = None
        if len(distances) > 0:
            min_distance = min(distances)
            if min_distance < tolerance:
                specific_person_index = distances.index(min_distance)

        for idx, (face_location, face_encoding) in enumerate(zip(face_locations, face_encodings)):
            top, right, bottom, left = face_location
            width = right - left
            height = bottom - top
            bbox = (left, top, width, height)

            # 얼굴 거리 계산
            distance = distances[idx]

            if distance < tolerance:
                if idx == specific_person_index:
                    name = "Specific Person"
                else:
                    name = "Candidate"
            else:
                name = "Unknown"

            # 추적기 초기화
            tracker = cv2.legacy.TrackerKCF_create()
            tracker.init(frame, bbox)
            trackers.append(tracker)
            face_names.append(name)
            face_distances.append(distance)

            # 얼굴 처리 및 주석 그리기
            process_face(frame, name, distance, bbox, frame_width, frame_height)

    else:
        # 모든 추적기 업데이트
        new_trackers = []
        new_face_names = []
        new_face_distances = []

        for tracker, name, distance in zip(trackers, face_names, face_distances):
            success, bbox = tracker.update(frame)
            if success:
                bbox = tuple(map(int, bbox))
                new_trackers.append(tracker)
                new_face_names.append(name)
                new_face_distances.append(distance)

                # 얼굴 처리 및 주석 그리기
                process_face(frame, name, distance, bbox, frame_width, frame_height)

        # 추적기와 얼굴 정보 업데이트
        trackers = new_trackers
        face_names = new_face_names
        face_distances = new_face_distances

    # 비디오 출력
    cv2.imshow('Video', frame)

    # 'q' 키를 누르면 종료
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

# 모든 작업 완료 후 클린업
video_capture.release()
cv2.destroyAllWindows()


In [1]:
import io
import cv2
import face_recognition
import numpy as np
from inswapper import FaceSwapper

base_image = "../face_recognition/test_hanni2.jpg"
source_image = "../face_recognition/faces/yunsy.jpg"
target_video = "hanni.mp4"

# 특정 인물의 이미지를 로드하여 인코딩
known_image = face_recognition.load_image_file(base_image)
known_face_encoding = face_recognition.face_encodings(known_image)[0]

# FaceSwapper 클래스 인스턴스 생성
face_swapper = FaceSwapper(det_size=(160, 160))

# 소스 얼굴 설정 (face_index는 선택 사항)
success = face_swapper.set_source_face(source_image)

if not success:
    print("소스 얼굴 설정에 실패했습니다.")
    exit()

def wrapper(bbox, frame_height, frame_width, func):

    left, top, width, height = bbox
    expand_ratio=0.3
    expand_width = int(width * expand_ratio)
    expand_height = int(height * expand_ratio)

    expanded_left = int(max(0, left - expand_width))
    expanded_top = int(max(0, top - expand_height))
    expanded_right = int(min(frame_width, left + width + expand_width))
    expanded_bottom = int(min(frame_height, top + height + expand_height))

    # 얼굴 영역 추출
    face_region = frame[expanded_top:expanded_bottom, expanded_left:expanded_right]

    if name == "Specific Person":
        # 얼굴 스왑 처리
        swap_image = func(face_region)
   
        # 스왑된 얼굴을 프레임에 적용
        frame[expanded_top:expanded_bottom, expanded_left:expanded_right] = swap_image
        color = (0, 0, 255)  # 빨간색
    elif name == "Candidate":
        color = (255, 0, 0)  # 파란색
    else:
        color = (0, 255, 0)  # 초록색

    # 사각형 그리기
    cv2.rectangle(frame, (expanded_left, expanded_top), (expanded_right, expanded_bottom), color, 2)
    # 이름과 거리 표시
    cv2.putText(frame, name, (expanded_left, expanded_bottom + 20), cv2.FONT_HERSHEY_SIMPLEX, 0.6, color, 2)
    cv2.putText(frame, f"Distance: {distance:.2f}", (expanded_left, expanded_bottom + 45), cv2.FONT_HERSHEY_SIMPLEX, 0.6, color, 2)

def process_face(face_region):
    swap_image = face_swapper.swap_faces_in_image(face_region)
    return swap_image if swap_image is not None else face_region

def view():

    # 비디오 파일 로드
    video_capture = cv2.VideoCapture(target_video)

    # 변수 초기화
    trackers = []
    face_names = []
    face_distances = []
    frame_skip = 12
    frame_count = 0

    while True:
        # 비디오에서 한 프레임씩 읽기
        ret, frame = video_capture.read()

        if not ret:
            break

        frame_count += 1

        rgb_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)

        # 프레임의 높이와 너비를 가져오기
        frame_height, frame_width, _ = frame.shape

        if frame_count % frame_skip == 0:
            # 추적기와 얼굴 정보 초기화
            trackers = []
            face_names = []
            face_distances = []

            # 얼굴 위치와 인코딩 탐지
            face_locations = face_recognition.face_locations(rgb_frame)
            face_encodings = face_recognition.face_encodings(rgb_frame, face_locations)

            distances = []
            for face_encoding in face_encodings:
                # 얼굴 거리 계산
                distance = face_recognition.face_distance([known_face_encoding], face_encoding)[0]
                distances.append(distance)

            # 임계값 설정 (예: 0.46)
            tolerance = 0.46

            # 거리 기준으로 가장 작은 인덱스 찾기
            specific_person_index = None
            if len(distances) > 0:
                min_distance = min(distances)
                if min_distance < tolerance:
                    specific_person_index = distances.index(min_distance)

            for idx, (face_location, face_encoding) in enumerate(zip(face_locations, face_encodings)):
                top, right, bottom, left = face_location
                width = right - left
                height = bottom - top
                bbox = (left, top, width, height)

                # 얼굴 거리 계산
                distance = distances[idx]

                if distance < tolerance:
                    if idx == specific_person_index:
                        name = "Specific Person"
                    else:
                        name = "Candidate"
                else:
                    name = "Unknown"

                # 추적기 초기화
                tracker = cv2.legacy.TrackerKCF_create()
                tracker.init(frame, bbox)
                trackers.append(tracker)
                face_names.append(name)
                face_distances.append(distance)

                # 얼굴 처리 및 주석 그리기
                wrapper(bbox, frame_height, frame_width, process_face)

        else:
            # 모든 추적기 업데이트
            new_trackers = []
            new_face_names = []
            new_face_distances = []

            for tracker, name, distance in zip(trackers, face_names, face_distances):
                success, bbox = tracker.update(frame)
                if success:
                    bbox = tuple(map(int, bbox))
                    new_trackers.append(tracker)
                    new_face_names.append(name)
                    new_face_distances.append(distance)

                    # 얼굴 처리 및 주석 그리기
                    wrapper(bbox, frame_height, frame_width, process_face)

            # 추적기와 얼굴 정보 업데이트
            trackers = new_trackers
            face_names = new_face_names
            face_distances = new_face_distances

        # 비디오 출력
        cv2.imshow('Video', frame)

        # 'q' 키를 누르면 종료
        if cv2.waitKey(1) & 0xFF == ord('q'):
            break

    # 모든 작업 완료 후 클린업
    video_capture.release()
    cv2.destroyAllWindows()

view()

c:\Users\prof\AppData\Local\Programs\Python\Python312\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
c:\Users\prof\AppData\Local\Programs\Python\Python312\Lib\site-packages\onnxruntime\capi\onnxruntime_inference_collection.py:69: UserWarning: Specified provider 'CUDAExecutionProvider' is not in available provider names.Available providers: 'AzureExecutionProvider, CPUExecutionProvider'
  warnings.warn(


Applied providers: ['CPUExecutionProvider'], with options: {'CPUExecutionProvider': {}}
find model: C:\Users\tiffanie.kim/.insightface\models\buffalo_l\1k3d68.onnx landmark_3d_68 ['None', 3, 192, 192] 0.0 1.0
Applied providers: ['CPUExecutionProvider'], with options: {'CPUExecutionProvider': {}}
find model: C:\Users\tiffanie.kim/.insightface\models\buffalo_l\2d106det.onnx landmark_2d_106 ['None', 3, 192, 192] 0.0 1.0
Applied providers: ['CPUExecutionProvider'], with options: {'CPUExecutionProvider': {}}
find model: C:\Users\tiffanie.kim/.insightface\models\buffalo_l\det_10g.onnx detection [1, 3, '?', '?'] 127.5 128.0
Applied providers: ['CPUExecutionProvider'], with options: {'CPUExecutionProvider': {}}
find model: C:\Users\tiffanie.kim/.insightface\models\buffalo_l\genderage.onnx genderage ['None', 3, 96, 96] 0.0 1.0
Applied providers: ['CPUExecutionProvider'], with options: {'CPUExecutionProvider': {}}
find model: C:\Users\tiffanie.kim/.insightface\models\buffalo_l\w600k_r50.onnx rec

c:\Users\prof\AppData\Local\Programs\Python\Python312\Lib\site-packages\insightface\utils\transform.py:68: FutureWarning: `rcond` parameter will change to the default of machine precision times ``max(M, N)`` where M and N are the input matrix dimensions.
To use the future default and silence this warning we advise to pass `rcond=None`, to keep using the old, explicitly pass `rcond=-1`.
  P = np.linalg.lstsq(X_homo, Y)[0].T # Affine matrix. 3 x 4


소스 얼굴이 설정되었습니다. 인덱스: 0


NameError: name 'frame' is not defined

: 

In [ ]:
source_file = "../face_recognition/faces/newjeans01_g.jpg"
known_file = "../face_recognition/test_hanni2.jpg"
target_file = "../face_recognition/test_yun.jpg"

import face_recognition
import cv2
import numpy as np

# 소스 이미지 로드 (얼굴을 교체할 이미지)
source_image = face_recognition.load_image_file(source_file)
source_image_rgb = source_image[:, :, ::-1]

# 대상 얼굴 이미지 로드 (교체할 얼굴)
target_face_image = face_recognition.load_image_file(target_file)
target_face_locations = face_recognition.face_locations(target_face_image)
if len(target_face_locations) == 0:
    print("대상 얼굴 이미지에서 얼굴을 찾을 수 없습니다.")
    exit()

# 대상 얼굴의 첫 번째 얼굴 선택
target_face_location = target_face_locations[0]
top_t, right_t, bottom_t, left_t = target_face_location
target_face = target_face_image[top_t:bottom_t, left_t:right_t]

# 교체할 얼굴 로드 (소스 이미지에서 교체하려는 특정 얼굴)
known_face_image = face_recognition.load_image_file(known_file)
known_face_encodings = face_recognition.face_encodings(known_face_image)
if len(known_face_encodings) == 0:
    print("교체할 얼굴 이미지에서 얼굴을 찾을 수 없습니다.")
    exit()
known_face_encoding = known_face_encodings[0]

# 소스 이미지에서 모든 얼굴과 인코딩 찾기
source_face_locations = face_recognition.face_locations(source_image)
source_face_encodings = face_recognition.face_encodings(source_image, source_face_locations)

# 소스 이미지의 각 얼굴에 대해 반복
for (face_location, face_encoding) in zip(source_face_locations, source_face_encodings):
    # 이 얼굴이 교체하려는 얼굴인지 확인
    matches = face_recognition.compare_faces([known_face_encoding], face_encoding, tolerance=0.4)

    if matches[0]:
        # 교체할 얼굴을 찾았습니다.
        top, right, bottom, left = face_location
        # 대상 얼굴을 교체할 얼굴의 크기에 맞게 조정
        face_width = right - left
        face_height = bottom - top
        target_face_resized = cv2.resize(target_face, (face_width, face_height))

        # 소스 이미지에서 얼굴 교체
        source_image[top:bottom, left:right] = target_face_resized

# 결과 저장
cv2.imwrite("result.jpg", cv2.cvtColor(source_image, cv2.COLOR_RGB2BGR))
